In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/amazonr/Prime_Pantry_5.json
/kaggle/input/dmusic/Digital_Music_5.json
/kaggle/input/mixedd/mixed5.csv
/kaggle/input/amazonrmix/Gift_Cards_5.json
/kaggle/input/amazonrmix/AMAZON_FASHION_5.json
/kaggle/input/amazonrmix/Luxury_Beauty_5.json
/kaggle/input/amazonrmix/Magazine_Subscriptions_5.json
/kaggle/input/glovetwitter27b100dtxt/glove.twitter.27B.100d.txt


In [2]:
import pandas as pd
import json
from nltk.corpus import stopwords
from string import punctuation
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input, Conv1D, GlobalMaxPool1D, Dropout, concatenate, Layer, InputSpec, LSTM
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras import activations, initializers, regularizers, constraints
from keras.utils.conv_utils import conv_output_length
from keras.regularizers import l2
from keras.constraints import maxnorm
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
import nltk, re, time

Using TensorFlow backend.


In [3]:
def parse(path):
  g = open(path)
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df1 = getDF('../input/dmusic/Digital_Music_5.json')
df2 = getDF('../input/amazonr/Prime_Pantry_5.json')
df3 = getDF('../input/amazonrmix/Luxury_Beauty_5.json')
df4 = getDF('../input/amazonrmix/Gift_Cards_5.json')
df5 = getDF('../input/amazonrmix/AMAZON_FASHION_5.json')
df6 = getDF('../input/amazonrmix/Magazine_Subscriptions_5.json')

In [4]:
f = [df1,df2, df3, df4, df5, df6]
df = pd.concat(f)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [5]:
df = df.reindex()

In [6]:
df = df[['overall','reviewText']]
df['reviewText'].fillna('',inplace=True)
df['overall'] = df['overall']/5.0

sentences = df['reviewText'].tolist()
labels = df['overall'].tolist()
len(sentences)
len(labels)

350370

In [7]:
len(labels)

350370

In [8]:

for i in range(len(labels)):
    if labels[i]>0.6:
        labels[i] = 3
    elif labels[i]>0.3:
        labels[i] = 2
    else:
        labels[i] = 1
labels.count(1)

6782

In [9]:
ohe = pd.get_dummies(labels)
labels = ohe
labels

,1,2,3
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
350365,1,0,0
350366,0,0,1
350367,0,0,1
350368,0,0,1


In [10]:
def clean_text(text, remove_stopwords=True):
    '''Clean the text, with the option to remove stopwords'''
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)
    
    # Clean the text
    text = re.sub(r"<br />", " ", text)
    text = re.sub(r"[^a-z]", " ", text)
    text = re.sub(r"   ", " ", text) # Remove any extra spaces
    text = re.sub(r"  ", " ", text)
    
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation])
    
    return(text)

data = []
for review in sentences:
    data.append(clean_text(review))

In [11]:
training_portion = .8
train_size = int(len(data) * training_portion)

train_clean = data[:train_size]
train_labels = labels[:train_size]

test_clean = data[train_size:]
test_labels = labels[train_size:]

print(len(train_clean))
print(len(train_labels))

280296
280296


In [12]:
# Tokenize the reviews
all_reviews = train_clean + test_clean
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_reviews)

train_seq = tokenizer.texts_to_sequences(train_clean)

test_seq = tokenizer.texts_to_sequences(test_clean)

In [13]:
word_index = tokenizer.word_index
print("Words in index: %d" % len(word_index))
max_review_length = 100

train_pad = pad_sequences(train_seq, maxlen = max_review_length)
test_pad = pad_sequences(test_seq, maxlen = max_review_length)

Words in index: 75059


In [14]:
X_train, x_valid, y_train, y_valid = train_test_split(train_pad, train_labels, test_size = 0.15, random_state = 2)
x_test = test_pad

In [15]:
print(x_valid.shape)
print(len(y_valid))

(42045, 100)
42045


In [16]:
EMBEDDING_FILE = '../input/glovetwitter27b100dtxt/glove.twitter.27B.100d.txt'
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

In [17]:
max_features = 20000
maxlen = 100

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
#change below line if computing normal stats is too slow
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size)) #embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
        

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3242: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


In [18]:
embedding_vector.shape

(100,)

In [19]:
weight_path="early_weights.hdf5"
checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=5)
callbacks = [checkpoint, early_stopping]

In [20]:
batch_size = 2048
epochs = 50
embed_size = 100

def lstm_model(conv_layers = 2, max_dilation_rate = 3):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
    x = Dropout(0.25)(x)
    x = Conv1D(2*embed_size, kernel_size = 3)(x)
    prefilt = Conv1D(2*embed_size, kernel_size = 3)(x)
    x = prefilt
    for strides in [1, 1, 2]:
        x = Conv1D(128*2**(strides), strides = strides, kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_size=3, kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10))(x)
    x_f = LSTM(512, kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10))(x)  
    x_b = LSTM(512, kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10))(x)
    x = concatenate([x_f, x_b])
    x = Dropout(0.5)(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(3, activation="softmax")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['categorical_accuracy'])

    return model

lstm_model = lstm_model()
lstm_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 100)     2000000     input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 100, 100)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 98, 200)      60200       dropout_1[0][0]                  
____________________________________________________________________________________________

In [21]:
lstm_model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, shuffle = True, validation_split=0.20, callbacks=callbacks)

/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 190600 samples, validate on 47651 samples
Epoch 1/50
190600/190600 [==============================] - 72s 379us/step - loss: 0.3468 - categorical_accuracy: 0.9089 - val_loss: 0.2997 - val_categorical_accuracy: 0.9180

Epoch 00001: val_loss improved from inf to 0.29968, saving model to early_weights.hdf5
Epoch 2/50
190600/190600 [==============================] - 66s 346us/step - loss: 0.2942 - categorical_accuracy: 0.9175 - val_loss: 0.2714 - val_categorical_accuracy: 0.9183

Epoch 00002: val_loss improved from 0.29968 to 0.27139, saving model to early_weights.hdf5
Epoch 3/50
190600/190600 [==============================] - 66s 346us/step - loss: 0.2715 - categorical_accuracy: 0.9194 - val_loss: 0.2545 - val_categorical_accuracy: 0.9234

Epoch 00003: val_loss improved from 0.27139 to 0.25446, saving model to early_weights.hdf5
Epoch 4/50
190600/190600 [==============================] - 66s 347us/step - loss: 0.2566 - categorical_accuracy: 0.9230 - val_loss: 0.2474 - val_catego

In [22]:
#6 domains into 3 categories : Good, Average, Bad
lstm_model.load_weights(weight_path)
score, acc = lstm_model.evaluate(x_test, test_labels, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

70074/70074 [==============================] - 7s 99us/step
Test score: 0.4356846055019377
Test accuracy: 0.8537545800209045
